In [1]:
import numpy as np
import sys
from pathlib import Path
import glob
import os

# option to import  from github folder
sys.path.insert(0, 'C:/Users/trose/Documents/GitHub/suite2p')
from suite2p import run_s2p

# set your options for running
ops = run_s2p.default_ops() # populates ops with the default options

In [2]:
#own imports
#scanimage load imports and file handling
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from pathlib import Path
from shutil import copy
#import pandas as pd

### PC file handling

In [121]:
# EXPERIMENT IDs
exp = {'61525','61526'}

main_root = 'I:/Joel Bauer/Data/imaging data/'
froot = 'E:/temp/trose/s2p_tiff/'      #temp drive for concatenating exp tiffs
fsave_tmp =  froot + '/suite2p'
fsave =  froot 
ftemp = 'C:/temp/trose/suite2ptemp'    #fast disk
ftemp_tmp = ftemp + '/suite2p'

Path(fsave_tmp).mkdir(parents=True, exist_ok=True)
Path(ftemp_tmp).mkdir(parents=True, exist_ok=True)

### Mac file handling

In [ ]:
# EXPERIMENT IDs
exp = {'62335','62336'}

froot = '/Users/trose/Data/temp/David Laubender/Data/imaging data/'
froot = '/Users/trose/Data/s2p_tiff/'
fsave_tmp =  froot + '/suite2p'
fsave =  froot 
ftemp = '/Users/trose/Data/temp/'
ftemp_tmp = ftemp + '/suite2p'

Path(fsave_tmp).mkdir(parents=True, exist_ok=True)
Path(ftemp_tmp).mkdir(parents=True, exist_ok=True)

In [ ]:
# Copy all tiffs to a temp folder for concatenated handling
files_fullpath = []
for val in exp:
    files = list(Path(main_root).rglob('exp'+val+'*.tif')) #recursive search over main_root
    for n,f in enumerate(files):
        #files[n] = os.path.basename(f)
        #files_fullpath.append(files[n])
        copy(files[n], froot)
        targetstring = [str(files[n]), str(froot)]
        print('copied {} to {}'.format(*targetstring))

copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_028.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_020.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_061.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_007.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_019.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_062.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_022.tif to E:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL

In [ ]:
files_fullpath = []
files_name = []

for val in exp:
    files = list(Path(froot).glob('exp'+val+'*.tif'))
    files = list(Path(froot).rglob('exp'+val+'*.tif')) #recursive
    #files = glob.glob(os.path.join(froot, 'exp'+val+'*.tif'))
    for n,f in enumerate(files):
        #files[n] = os.path.basename(f)
        files_fullpath.append(str(files[n]))
        files_name.append(f.name)

    
files_fullpath = sorted(files_fullpath)
files_name = sorted(files_name)

In [ ]:
with ScanImageTiffReader(files_fullpath[0]) as reader:
    header = (reader.description(0))
    mov_dim = (reader.shape())
    
level = pSI.parse_SI_header_level(header)
zoom = pSI.parse_SI_header_zoom(header)
framerate = pSI.parse_SI_header_FrameRate(header)
channels = pSI.parse_SI_header_Channels(header)
volumes = pSI.parse_SI_header_Volumes(header)
frames = pSI.parse_SI_header_Frames(header)
frames_per_file = pSI.parse_SI_header_FramesPerFile(header)

# account for multilevel acq where frames is 1
if frames < volumes:
    frames = volumes

In [ ]:
bad_frames = np.linspace(0, 49, 50)
np.save(fsave_tmp + '/bad_frames.npy', bad_frames)
np.save(ftemp_tmp + '/bad_frames.npy', bad_frames)
#np.save(ftemp + '/bad_frames.npy', bad_frames)

In [ ]:
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data', # file paths
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': [froot],  # a list of folders with tiffs
                             # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
      'delete_bin': False,
      'save_path0': fsave,
      'fast_disk': ftemp,
      'subfolders': [], # choose subfolders of 'data_path' to look in (optional)

      # main settings
      'nplanes': level, # each tiff has these many planes in sequence
      'nchannels': channels, # each tiff has these many channels per plane
      'functional_chan': 1, # this channel is used to extract functional ROIs (1-based)
      'tau':  1., # this is the main parameter for deconvolution
      'fs': framerate / level,  # sampling rate (PER PLANE - e.g. if you have 12 planes then this should be around 2.5)
      'preclassify': 0, # apply classifier before signal extraction with probability 0.5 (turn off with value 0)
      'frames_include:': -1,  #default: -1) if greater than zero, only frames_include frames are processed. useful for testing parameters on a subset of data.
      # output settings
      'save_mat': True, # whether to save output as matlab files
      'combined': True, # combine multiple planes into a single result /single canvas for GUI

      'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
      'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
      'force_sktiff': False,
      # bidirectional phase offset
      'do_bidiphase': True,
      'bidiphase': 0,

      # registration settings
      'do_registration': 1, # whether to register data (2 forces re-registration)
      'two_step_registration:': True, #default: False) whether or not to run registration twice (for low SNR data). keep_movie_raw must be True for this to work.
      'keep_movie_raw': True,
      'nimg_init': 500, # subsampled frames for finding reference image
      'batch_size': 5000, # number of frames per batch
      'maxregshift': 0.05, # max allowed registration shift, as a fraction of frame max(width and height)
      'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
      'reg_tif': True, # whether to save registered tiffs
      'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
      'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
      'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
      'smooth_sigma_time': 1, # default: 0) standard deviation in time frames of the gaussian used to smooth the data before phase correlation is computed. Might need this to be set to 1 or 2 for low SNR data.
      'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
      'pad_fft': True,

      # non rigid registration settings
      'nonrigid': True, # whether to use nonrigid registration
      'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
      'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
      'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid

      # cell detection settings
      'roidetect': True, # whether or not to run ROI extraction
      'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
      'diameter': [9,12], #this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
      'connected': False, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
      'nbinned': 5000, # max number of binned frames for cell detection
      'max_iterations': 50, # maximum number of iterations to do cell detection
      'threshold_scaling': 2.0, # adjust the automatically determined threshold by this scalar multiplier
      'max_overlap': 0.9, # cells with more overlap than this get removed during triage, before refinement
      'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
      'smooth_masks': True, # default: True) whether to smooth masks in final pass of cell detection. This is useful especially if you are in a high noise regime.

      # ROI extraction parameters
      'sparse_mode': False, #default: False) whether or not to use sparse_mode cell detection
      'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
      'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
      'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)

      # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
      'chan2_thres': 0.65, # minimum for detection of brightness on channel 2

      # deconvolution settings
      'baseline': 'maximin', # baselining mode (can also choose 'prctile')
      'win_baseline': 60., # window for maximin
      'sig_baseline': 10., # smoothing constant for gaussian filter
      'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
      'neucoeff': .7,  # neuropil coefficient


      # List of tiffs to be loaded
      'tiff_list': files_name # list of tiffs in folder * data_path *!
}


In [110]:
opsEnd = run_s2p.run_s2p(ops=ops, db=db)

{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['E:/temp/trose/s2p_tiff/'], 'delete_bin': False, 'save_path0': 'E:/temp/trose/s2p_tiff/', 'fast_disk': 'C:/temp/trose/suite2ptemp', 'subfolders': [], 'nplanes': 4, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 7.5, 'preclassify': 0, 'frames_include:': -1, 'save_mat': True, 'combined': True, 'num_workers': 0, 'num_workers_roi': 0, 'force_sktiff': False, 'do_bidiphase': True, 'bidiphase': 0, 'do_registration': 1, 'two_step_registration:': True, 'keep_movie_raw': True, 'nimg_init': 500, 'batch_size': 5000, 'maxregshift': 0.05, 'align_by_chan': 1, 'reg_tif': True, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma': 1.15, 'smooth_sigma_time': 1, 'th_badframes': 1.0, 'pad_fft': True, 'nonrigid': True, 'block_size': [128, 128], 'snr_thresh': 1.2, 'maxregshiftNR': 5, 'roidetect': True, 'spatial_scale': 0, 'diameter': [9, 12], 'connected': False, 'nbinned': 5000, 'max_iterations': 50, 'threshold_scaling':

C:\Users\trose\AppData\Local\Continuum\miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 2.42 sec.
Extracted fluorescence from 175 ROIs in 1250 frames, 2.75 sec.
----------- Total 30.15 sec.
----------- SPIKE DECONVOLUTION
----------- Total 26.81 sec.
Plane 0 processed in 153.45 sec (can open in GUI).
total = 161.75 sec.
>>>>>>>>>>>>>>>>>>>>> PLANE 1 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 1250 frames
NOTE: estimated bidiphase offset from data: 1 pixels
Reference frame, 33.51 sec.
1250/1250 frames, 62.35 sec.
bad frames file path: E:\temp\trose\s2p_tiff\bad_frames.npy
----------- Total 96.40 sec
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 08
Binned movie [154,464,462], 1.60 sec.
ROIs: 150, cost: 0.5629, time: 8.1123
ROIs: 185, cost: 0.5585, time: 10.1779
ROIs: 185, cost: 0.5581, time: 11.5024
Binning movie in chunks of length 08
Binned movie [154,464,462], 1.39 sec.
ROIs: 185, cost: 0.5868, time: 13.9149
ROIs: 185, cost: 0.5771, time: 14.9447
ROIs: 185, cost: 0.5765, time: 15.7599
Found 185 ROIs, 18.66 sec
NO

C:\Users\trose\AppData\Local\Continuum\miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 2.39 sec.
Extracted fluorescence from 185 ROIs in 1250 frames, 2.79 sec.
----------- Total 30.23 sec.
----------- SPIKE DECONVOLUTION
----------- Total 26.34 sec.
Plane 1 processed in 153.13 sec (can open in GUI).
total = 314.88 sec.
>>>>>>>>>>>>>>>>>>>>> PLANE 2 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 1250 frames
NOTE: estimated bidiphase offset from data: 1 pixels


KeyboardInterrupt: 

In [ ]:
stat = np.load(ftemp +'/std_ops/ops_dendrite.npy', allow_pickle=True)

In [ ]:
# set your options for running
ops = run_s2p.default_ops() # populates ops with the default options

In [ ]:
ftemp

In [ ]:
opspath = 'C:/temp/trose/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/suite2p/suite2p/plane0/ops.npy'

stato = np.load(opspath, allow_pickle=True)

In [ ]:
print(stat.conj().transpose())
